In [665]:
import pandas as pd
import json
import glob
import re

In [666]:
csv_files = glob.glob('./output/target_chain_in_usdc/*.csv')
dfs = [pd.read_csv(file) for file in csv_files]
df = pd.concat(dfs, ignore_index=True)

In [667]:
df['response'] = df['response'].apply(json.loads)

In [668]:
def extract_attribute(response, keys):
    message = response.get("message")
    
    if message:
        if re.search(r"The chain with ID \d+ is currently disabled", message):
            return 'disabled'
        if re.search(r"Could not find token USDC on chain \d+", message):
            return 'no_token'
        elif message == "No available quotes for the requested transfer":
            return 'error'
        elif message == "The same token cannot be used as both the source and destination token.":
            return 'exclude'
    
    result = response
    for key in keys:
        if isinstance(result, dict):
            result = result.get(key)
        elif isinstance(result, list) and isinstance(key, int):
            if key < len(result):
                result = result[key]
            else:
                return None
        else:
            return None
    return result

In [669]:
attributes = [
    ('bridge', ['toolDetails', 'name']),
    ('source_chain', ['action', 'fromChainId']),
    ('sent_token', ['action', 'fromToken', 'symbol']),
    ('sent_amount', ['action', 'fromAmount']),
    ('target_chain', ['action', 'toChainId']),
    ('recived_token', ['action', 'toToken', 'symbol']),
    ('est_recived_amount', ['estimate', 'toAmount']),
    ('est_tx_fee', ['estimate', 'feeCosts', 0, 'amountUSD']),
    ('router_fee_usd', ['estimate', 'feeCosts', 0, 'amountUSD']),
    ('relayer_fee_usd', ['estimate', 'feeCosts', 1, 'amountUSD']),
    ('est_gas_cost_usd', ['estimate', 'gasCosts', 0, 'amountUSD']),
    ('gas_symbol', ['estimate', 'gasCosts', 0, 'token', 'symbol']),
    ('est_amount_sent_usd', ['estimate', 'fromAmountUSD']),
    ('est_amount_recived_usd', ['estimate', 'toAmountUSD'])
]

for col_name, keys in attributes:
    df[col_name] = df['response'].apply(lambda x: extract_attribute(x, keys))

df['action_present'] = df['response'].apply(lambda x: 'action' in x)

In [670]:
chain_id_labels = {
    1: 'Ethereum',
    10: 'Optimism',
    25: 'Cronos',
    56: 'Binance',
    66: 'OKEX',
    100: 'Gnosis',
    106: 'Velas',
    122: 'Fuse',
    137: 'Polygon',
    250: 'Fantom',
    288: 'Boba',
    324: 'zkSync Era',
    1101: 'Polygon zkEVM',
    1284: 'Moonbeam',
    1285: 'Moonriver',
    8453: 'Base',
    9001: 'Evmos',
    42161: 'Arbitrum',
    42220: 'Celo',
    43114: 'Avalanche',
    59144: 'Linea',
    1313161554: 'Aurora',
    1666600000: 'Harmony'
}

def replace_with_labels(chain_id):
    return [chain_id_labels.get(chain, chain) for chain in chain_id]

df['source_chain'] = df['source_chain'].replace(chain_id_labels)
df['target_chain'] = df['target_chain'].replace(chain_id_labels)

In [671]:
df = df.drop(['response'], axis=1)
df = df[df['bridge'] != 'exclude']

df.reset_index(drop=True, inplace=True)

In [672]:
df.to_csv('./output/usdc_transfer_quotes.csv', index=False)